# Test Triangulation Calculation Functions

This notebook allows interactive testing of the functions from triangulationCalculation.py

In [1]:
import os
import sys

# Get the current working directory
current_dir = os.getcwd()

# Define the relative path to your target folder or file
relative_path = "src/"

# Construct the full path
local_path = os.path.join(current_dir, relative_path)
print(local_path)

sys.path.insert(0, local_path)

# Import all required libraries and functions
from src.triangulationCalculation import *
import pandas as pd
import numpy as np
import json
import folium
from IPython.display import display, HTML

/Users/c/Desktop/project/ipvote_2024/ipvote/serverless/triangulation/calculation/src/
[0] Running on macOS - attempting to load local AWS credentials
[0] Successfully loaded AWS credentials from local file


## Test Individual Functions

In [4]:
# Test calculate_max_distance
print("Testing calculate_max_distance:")
test_times = [10, 50, 100, 200]
for time_ms in test_times:
    min_dist, max_dist = calculate_max_distance(time_ms)
    print(f"Time: {time_ms}ms -> Min distance: {min_dist:.2f}km, Max distance: {max_dist:.2f}km")

Testing calculate_max_distance:
Time: 10ms -> Min distance: 0.00km, Max distance: 0.00km
Time: 50ms -> Min distance: 3997.23km, Max distance: 7267.70km
Time: 100ms -> Min distance: 8993.77km, Max distance: 16352.32km
Time: 200ms -> Min distance: 18986.86km, Max distance: 34521.56km


In [4]:
# Test get_world_boundaries
print("Testing get_world_boundaries:")
world_geometries = get_world_boundaries()
print(f"Number of countries loaded: {len(world_geometries)}")
print("Sample of country codes:", list(world_geometries.keys())[:5])

Testing get_world_boundaries:
Number of countries loaded: 239
Sample of country codes: ['AW', 'AF', 'AO', 'AI', 'AL']


In [8]:
# Test create_map with sample data
print("Testing create_map:")

# Sample data
test_min_distances = {
    'us-east-1': 1000,
    'eu-west-1': 800
}

test_min_latencies = {
    'us-east-1': 50,
    'eu-west-1': 40
}

# Create some sample points
test_points = [
    [-77, 39],  # Near us-east-1
    [-6, 53]    # Near eu-west-1
]
test_weights = [0.8, 0.6]

# Create and display the map
m = create_map(DATACENTERS, test_min_distances, test_min_latencies, test_points, test_weights)
display(m)

Testing create_map:


In [9]:
# Test get_intersecting_countries
print("Testing get_intersecting_countries:")

# Create a sample polygon (rough rectangle over Western Europe)
test_polygon = [
    [0, 45],   # Bottom-left
    [10, 45],  # Bottom-right
    [10, 55],  # Top-right
    [0, 55],   # Top-left
    [0, 45]    # Close the polygon
]

intersecting = get_intersecting_countries(test_polygon)
print("Intersecting countries:", intersecting)

Testing get_intersecting_countries:
Intersecting countries: ['Austria', 'Belgium', 'Switzerland', 'Germany', 'Denmark', 'France', 'United Kingdom', 'Italy', 'Liechtenstein', 'Luxembourg', 'Netherlands']


## Test Complete Lambda Handler

Create a sample event to test the full lambda_handler function

In [2]:
# Create a sample event
test_event = {
    'requestContext': {
        'http': {
            'sourceIp': '2001:920:3812:4a01:98ae:2471:6e91:27ab'  # Example IP
        }
    }
}

# Test lambda_handler
response = lambda_handler(test_event, None)

print("Response status code:", response['statusCode'])
if 'X-Possible-Countries' in response.get('headers', {}):
    print("Possible countries:", response['headers']['X-Possible-Countries'])

# Display the generated map
if response['statusCode'] == 200:
    display(HTML(response['body']))

[1] Starting processing for Target IP: 2001:920:3812:4a01:98ae:2471:6e91:27ab
[1.2] Getting recent files from S3
[1.3] Searching S3 with prefix: triangulation/2001:920:3812:4a01:98ae:2471:6e91:27ab/
[1.4] Processed 10 files...
[1.4] Processed 20 files...
[1.4] Processed 30 files...
[1.4] Processed 40 files...
[1.4] Processed 50 files...
[1.6] Completed S3 search. Processed 54 files, found 54 relevant records
[2] Retrieved 54 records from S3
[3] Starting clock shift calculations for 12 unique nonces
[4] Calculated median clock shift: -37905.75 ms
[5] Found 5 valid nonces out of 6
[6] Starting minimum latency calculations

[7] Calculated minimum latencies:
  eu-central-1: 17.50 ms
  ap-northeast-1: 126.50 ms
  ap-south-1: 81.50 ms
  eu-west-1: 23.50 ms
  sa-east-1: 108.50 ms
  us-east-1: 64.50 ms
  us-west-2: 87.50 ms
  af-south-1: 99.00 ms

[8] Starting point generation with 175 points
Error creating convex hull: TopologyException: side location conflict at -134.49907046614044 58.335821